# Dịch máy bằng các cách kết hợp 

Ta sẽ lấy embedding của BERT  sau đso tiến hành thêm 1 lớp RNN để tăng tính liên kết, cuối cùng là đưa ra dự đoán cho dữ liệu 

In [4]:
import torch 

import random 
import numpy as np 

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True # hàm này hay dùng để đảm bảo mỗi lần chạy đều ra kết quả giống nhau

Ta sẽ phải mã hóa các như như cách Bert_pretrain mã hóa, trong trường hợp này ta sử dụng BertTokenizer 


In [15]:
from transformers import BertTokenizer, BertModel 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') 

In [16]:
tokenizer.vocab_size

30522

In [17]:
test_sentence = "this is a test sentence"
tokenized = tokenizer.tokenize(test_sentence)
tokenized

['this', 'is', 'a', 'test', 'sentence']

Ta còn có thể số hóa các từ trong vocab bằng cách sử dụng tokenizer.convert_tokens_to_ids

In [18]:
tokenizer.convert_tokens_to_ids(tokenized)

[2023, 2003, 1037, 3231, 6251]

bert được huấn luyện với các ký tự đặc biệt: bắt đầu và kết thúc, các ký tự padding (khi độ dài câu ngắn) và unknown (khi từ không có trong vocab)

tokenizer có thuộc tính bắt đầu và kết thúc sequence (bos_token và eos_token) tuy nhiên không nên sử dụng 

In [20]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token
print(init_token, eos_token, pad_token, unk_token)


[CLS] [SEP] [PAD] [UNK]


In [21]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


kiểm tra độ  dài của câu mà model pre tráin đã sử dụng để phục vụ xử  lý dữ liệu tiếp theo 


In [24]:
max_input_length =  tokenizer.max_model_input_sizes["bert-base-uncased"]
print(max_input_length) # 512


512


chúng ta cần một hàm truyền vào trường TEXT để thực hiện tokenize. Hàm này thực hiện cắt ngắn câu nếu câu đã cho vượt quá max_length, do chúng ta cần thực hiện thêm 2 token đầu và cuối vào chuỗi nên thực tế độ dài câu ta thu được phải là max_length - 2 


In [ ]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) # tokenize the sentence 
    tokens = tokens[:max_input_length-2] # cut to max_input_length
    return tokens


